<a href="https://colab.research.google.com/github/cs145442/nlp-projects-with-tf2/blob/master/bert_on_natural_question.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

try, resolving dependencies with **tf_v1**

In [1]:
%tensorflow_version 1.15.2

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.15.2`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.


In [2]:
import tensorflow
print(tensorflow.__version__)

1.15.2


**Using BERT on NQ**

[git/google-research](https://github.com/google-research/language/tree/master/language/question_answering/bert_joint)

In [3]:
!pip install --no-dependencies bert-tensorflow natural-questions

In [4]:
!gsutil cp -R gs://bert-nq/bert-joint-baseline .

Copying gs://bert-nq/bert-joint-baseline/bert_config.json...
Copying gs://bert-nq/bert-joint-baseline/bert_joint.ckpt.data-00000-of-00001...
Copying gs://bert-nq/bert-joint-baseline/bert_joint.ckpt.index...
Copying gs://bert-nq/bert-joint-baseline/nq-train.tfrecords-00000-of-00001...
\ [4 files][  2.3 GiB/  2.3 GiB]   65.7 MiB/s                                   
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying gs://bert-nq/bert-joint-baseline/vocab-nq.txt...
\ [5 files][  2.3 GiB/  2.3 GiB]   64.9 MiB/s                                   
Operation completed over 5 objects/2.3 GiB.                                      


In [5]:
!ls bert-joint-baseline/

bert_config.json		     nq-train.tfrecords-00000-of-00001
bert_joint.ckpt.data-00000-of-00001  vocab-nq.txt
bert_joint.ckpt.index


In [6]:
!git clone https://github.com/google-research/language.git

fatal: destination path 'language' already exists and is not an empty directory.


In [7]:
! python language/setup.py install --prefix=~

/usr/local/lib/python3.6/dist-packages/setuptools/dist.py:454: UserWarning: Normalizing '0.0.1.dev' to '0.0.1.dev0'
  warnings.warn(tmpl.format(**locals()))
running install
Checking .pth file support in /root/lib/python3.6/site-packages/
/usr/bin/python3 -E -c pass
TEST FAILED: /root/lib/python3.6/site-packages/ does NOT support .pth files
bad install directory or PYTHONPATH

You are attempting to install a package to a directory that is not
on PYTHONPATH and which Python does not read ".pth" files from.  The
installation directory you specified (via --install-dir, --prefix, or
the distutils default setting) was:

    /root/lib/python3.6/site-packages/

and your PYTHONPATH environment variable currently contains:

    '/tensorflow-1.15.2/python3.6:/env/python'

Here are some of your options for correcting the problem:

* You can choose a different installation directory, i.e., one that is
  on PYTHONPATH or supports .pth files

* You can add the installation directory to the PYTHONPATH

*this step is for **data preparation**, since we've already downloaded the pretrained data*
*we'll test the data on dev dataset*
Intrestingly, with **tf_v1**, data prep step worked fine! 



In [0]:
# ! cd language && python -m language.question_answering.bert_joint.prepare_nq_data \
#   --logtostderr \
#   --input_jsonl ~/data/nq-train-??.jsonl.gz \
#   --output_tfrecord ~/output_dir/nq-train.tfrecords-00000-of-00001 \
#   --max_seq_length=512 \
#   --include_unknowns=0.02 \
#   --vocab_file=../bert-joint-baseline/vocab-nq.txt

In [13]:
# getting a tiny dev set to verify as trusted
! gsutil cp -R gs://bert-nq/tiny-dev .

Copying gs://bert-nq/tiny-dev/nq-dev-sample.jsonl.gz...
Copying gs://bert-nq/tiny-dev/nq-dev-sample.no-annot.jsonl.gz...
\ [2 files][ 50.2 MiB/ 50.2 MiB]                                                
Operation completed over 2 objects/50.2 MiB.                                     


In [17]:
! cd language && python -m language.question_answering.bert_joint.run_nq \
  --logtostderr \
  --bert_config_file=../bert-joint-baseline/bert_config.json \
  --vocab_file=../bert-joint-baseline/vocab-nq.txt \
  --predict_file=../tiny-dev/nq-dev-sample.no-annot.jsonl.gz \
  --init_checkpoint=../bert-joint-baseline/bert_joint.ckpt \
  --do_predict \
  --output_dir=bert_model_output \
  --output_prediction_file=bert_model_output/predictions.json

# ! python -m natural_questions.nq_eval \
#   --logtostderr \
#   --gold_path=tiny-dev/nq-dev-sample.jsonl.gz \
#   --predictions_path=bert_model_output/predictions.json



W0608 09:15:30.156173 140514038568832 module_wrapper.py:139] From /usr/local/lib/python3.6/dist-packages/bert/modeling.py:93: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

W0608 09:15:30.243681 140514038568832 estimator.py:1994] Estimator's model_fn (<function model_fn_builder.<locals>.model_fn at 0x7fcb95ceb620>) includes params argument, but params are not passed to Estimator.
INFO:tensorflow:Using config: {'_model_dir': 'bert_model_output', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_ses

**Updates**

*didn't work out!*

*most of the research code is on tf_v1 whereas current practise is available on tf_v2, too much dependencies to resolve*

*restarting the runtime, with tf_v1*

*worked fine*

In [41]:
cd language/

/content/language


In [46]:
import numpy as np
import pandas as pd
import tensorflow as tf
# import tf2_0_baseline_w_bert as tf2baseline # old script
import tf2_0_baseline_w_bert_translated_to_tf2_0 as tf2baseline # Oliviera's script
import bert_modeling as modeling
import bert_optimization as optimization
import bert_tokenization as tokenization
import json
import absl
import sys

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

ModuleNotFoundError: ignored

In [64]:
# ! ls
# ! python setup.py install --prefix=~
# # setup complete
# ! cd ..
# % cd ..
! ls/bert-joint-baseline

/bin/bash: ls/bert-joint-baseline: No such file or directory


In [66]:
! ls bert-joint-baseline


bert_config.json		     nq-train.tfrecords-00000-of-00001
bert_joint.ckpt.data-00000-of-00001  vocab-nq.txt
bert_joint.ckpt.index
